In [1]:
# 用以下指令連結spark
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions

spark = SparkSession.builder.appName('instacart').getOrCreate()

In [2]:
# 用SparkSession讀取csv為DataFrame
prior_order = spark.read.csv(header=True, path='./instacart_2017_05_01/order_products__prior.csv')
order = spark.read.csv(header=True, path='./instacart_2017_05_01/orders.csv')
train = spark.read.csv(header=True, path='./instacart_2017_05_01/order_products__train.csv')
product = spark.read.csv(header=True, path='./instacart_2017_05_01/products.csv')

In [3]:
# 觀察資料
prior_order.take(20)

[Row(order_id='2', product_id='33120', add_to_cart_order='1', reordered='1'),
 Row(order_id='2', product_id='28985', add_to_cart_order='2', reordered='1'),
 Row(order_id='2', product_id='9327', add_to_cart_order='3', reordered='0'),
 Row(order_id='2', product_id='45918', add_to_cart_order='4', reordered='1'),
 Row(order_id='2', product_id='30035', add_to_cart_order='5', reordered='0'),
 Row(order_id='2', product_id='17794', add_to_cart_order='6', reordered='1'),
 Row(order_id='2', product_id='40141', add_to_cart_order='7', reordered='1'),
 Row(order_id='2', product_id='1819', add_to_cart_order='8', reordered='1'),
 Row(order_id='2', product_id='43668', add_to_cart_order='9', reordered='0'),
 Row(order_id='3', product_id='33754', add_to_cart_order='1', reordered='1'),
 Row(order_id='3', product_id='24838', add_to_cart_order='2', reordered='1'),
 Row(order_id='3', product_id='17704', add_to_cart_order='3', reordered='1'),
 Row(order_id='3', product_id='21903', add_to_cart_order='4', reor

In [4]:
# 呈現表格
prior_order.filter(prior_order['order_id'] == '3').show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|
+--------+----------+-----------------+---------+



In [5]:
# 計算數量
print(order.select('user_id').distinct().count()) # 計算總訂單表內客戶數量

# 計算總訂單表內不屬於prior表的訂單，表示每個人最後一張訂單都被取出作為test或train
print(order.filter(order['eval_set'] != 'prior').count())

206209
206209


In [6]:
# 找出最多訂單的前幾名客戶

order.groupBy('user_id').count().sort("count", ascending=False).show(15)

+-------+-----+
|user_id|count|
+-------+-----+
|   8832|  100|
|  31118|  100|
| 121370|  100|
| 183981|  100|
|  86804|  100|
| 111982|  100|
|  69182|  100|
|  81549|  100|
|  77965|  100|
|  88150|  100|
| 166908|  100|
|  81342|  100|
| 103206|  100|
|  42975|  100|
| 101261|  100|
+-------+-----+
only showing top 15 rows



In [7]:
print(order.first())
print(prior_order.first())
print(train.first())
print(product.first())

Row(order_id='2539329', user_id='1', eval_set='prior', order_number='1', order_dow='2', order_hour_of_day='08', days_since_prior_order=None)
Row(order_id='2', product_id='33120', add_to_cart_order='1', reordered='1')
Row(order_id='1', product_id='49302', add_to_cart_order='1', reordered='1')
Row(product_id='1', product_name='Chocolate Sandwich Cookies', aisle_id='61', department_id='19')


In [8]:
# 合併prior和train訂單資料，生成客戶的偏好表

# 把test的訂單去除，並刪除目前不需要的欄位
preference = order.filter(order['eval_set'] != 'test')
preference = preference.drop('order_dow').drop('order_hour_of_day').drop('days_since_prior_order').drop('order_number')

# 合併prior和train訂單資料
prefer_prior = preference.join(prior_order, "order_id", "inner").select("user_id", "order_id", "product_id")
prefer_train = preference.join(train, "order_id", "inner").select("user_id", "order_id", "product_id")
preference = prefer_prior.unionAll(prefer_train)

In [9]:
# 查詢特定客戶購買的產品數量

preference.filter(preference['user_id']=='106510').groupBy("product_id").count().show()

+----------+-----+
|product_id|count|
+----------+-----+
|      9387|   25|
|     39877|   27|
+----------+-----+



In [10]:
# 加上產品名稱

preference = preference.join(product, "product_id").select("user_id", "order_id", "product_id", "product_name")

In [11]:
# 重新查詢客戶購買的產品

preference.filter(preference['user_id']=='106510').groupBy("product_name").count().collect()

[Row(product_name='Organic Granny Smith Apple', count=27),
 Row(product_name='Granny Smith Apples', count=25)]